# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-7fb80672bf-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the common aspects between movies directed by Woody Allen or Quentin Tarantino. We want to know the people that worked for both directors with some numerical analyses, what are the differences in terms of budget for their movies, who won more Academy Awards. 

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q2526255`| Director       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`    | Quentin Tarantino      | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**


The workload should:


1. Identify the BGP for films

2. Identify the BGP for directors

3. Identify the BGP for workers in a films

4. Compare the workers amongst the films directed by the two directors

5. Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

6. Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

7. Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

    7.1 Find the BGP for Academy Awards 

    7.2 Find the related subproperties

    7.3 Find how they are related to the directors
    
    7.4 Are there alternative queries to get the same result?



## Task 1

In [3]:
#let's start with tarantino and retrieve all the properties associated
q = f"""
select distinct ?property ?name where{{
wd:Q3772 ?property ?obj .
?property <http://schema.org/name> ?name
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P8585'), ('name', 'FIPRESCI person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('property', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('pr

186

In [5]:
#let's inspect property "filmography"
q = f"""
select distinct ?movie ?name where{{
wd:Q3772 wdt:P1283 ?movie .
?movie <http://schema.org/name> ?name
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q6794012'), ('name', 'Quentin Tarantino filmography')]


1

In [6]:
#let's see the properties of Tarantino's Filmography
Filmography_Tarantino = 'wd:Q6794012'
q = f"""
select distinct ?prop ?name where{{
wd:Q6794012 ?prop ?obj .
?prop <http://schema.org/name> ?name
}}
"""
run_query(q)

[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('prop', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]


5

In [8]:
#maybe we have to look for movies directed by tarantino, not movies that tarantino directed
#basically, look for subjects for which tarantino is the object

#tarantino directed Kill Bill, let's try and filter by name of the movie
q = f"""
select distinct ?movie ?name where{{
?movie ?property wd:Q3772 .
?movie <http://schema.org/name> ?name .
filter regex(?name, '.*kill bill.*', 'i')
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q223132'), ('name', 'Kill Bill: The Whole Bloody Affair')]
[('movie', 'http://www.wikidata.org/entity/Q165325'), ('name', 'Kill Bill Volume 1')]
[('movie', 'http://www.wikidata.org/entity/Q4382298'), ('name', 'Kill Bill Vol. 2 – Original Soundtrack')]
[('movie', 'http://www.wikidata.org/entity/Q928019'), ('name', 'Kill Bill Volume 2')]
[('movie', 'http://www.wikidata.org/entity/Q18126825'), ('name', 'Kill Bill: The Whole Bloody Affair')]


5

In [9]:
#let's pick up the node relative to Kill Bill Volume 1 and let's figure out first of all its class
KillBill = 'wd:Q165325'
q = f"""
select distinct ?class ?name where{{
wd:Q165325 wdt:P31 ?class .
?class <http://schema.org/name> ?name
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]


1

## Task 3

In [10]:
Film = 'wd:Q11424'
#trying to figure out all of the properties relative to all of the movies of wikidata would be useless
#let's try and find the properties exiting kill bill first, hopefully we will be able to find
#properties for director, workers in a film and so on
q = f"""
select distinct ?property ?name where{{
wd:Q165325 ?property ?obj .
?property <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P7777'), ('name', 'AdoroCinema film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8918'), ('name', 'Max Movie film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('property', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]

130

In [11]:
editor = 'wdt:P1040'
castMember = 'wdt:P161'
genre = 'wdt:P136'
producer = 'wdt:P162'
awardReceived = 'wdt:P166'
boxOffice = 'wdt:P2142'
cost = 'wdt:P2130'
composer = 'wdt:P86'
publicationDate = 'wdt:P577'

#there's a lot of properties, some we isolated through inspection, but there's a lot.
#let's use tarantino as object to try and identify useful properties involving kill bill and tarantino
q = f"""
select distinct ?property ?name where{{
wd:Q165325 ?property wd:Q3772 .
?property <http://schema.org/name> ?name
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P57'), ('name', 'director')]
[('property', 'http://www.wikidata.org/prop/direct/P58'), ('name', 'screenwriter')]


3

In [13]:
#let's answer point 1. to get all movies directed by tarantino we could write
q = f"""
select distinct ?movie ?name ?date where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    <http://schema.org/name> ?name ;
    wdt:P577 ?date .

}}
order by asc(?date)
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q72962'), ('name', 'Reservoir Dogs'), ('date', '1992-09-10T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q72962'), ('name', 'Reservoir Dogs'), ('date', '1992-10-23T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction'), ('date', '1994-05-21T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction'), ('date', '1994-10-14T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction'), ('date', '1994-11-03T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q431252'), ('name', 'Jackie Brown'), ('date', '1997-01-01T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q431252'), ('name', 'Jackie Brown'), ('date', '1997-12-25T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q431252'), ('name', 'Jackie Brown'), ('date', '1998-04-16T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q165325'), ('name', 'Kill Bill Volume 1'), 

28

In [14]:
#repetitions are probably publication dates in various continents/states, we just care about the movies for now
#but it's nice to have them ordered by publication

#let's do the same for Woody Allen
q = f"""
select distinct ?movie ?name ?date where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    <http://schema.org/name> ?name ;
    wdt:P577 ?date .

}}
order by asc(?date)
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?"), ('date', '1966-01-01T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q1146315'), ('name', 'Take the Money and Run'), ('date', '1969-01-01T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q806092'), ('name', 'Bananas'), ('date', '1971-01-01T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q499697'), ('name', 'Everything You Always Wanted to Know About Sex*'), ('date', '1972-01-01T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q750003'), ('name', 'Sleeper'), ('date', '1973-12-17T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q750003'), ('name', 'Sleeper'), ('date', '1974-04-25T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q806092'), ('name', 'Bananas'), ('date', '1974-08-30T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q1218888'), ('name', 'Love and Death'), ('date', '1975-01-01T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q23346

101

In [15]:
#the property to answer point 2 was already found
#to find the director of Kill Bill we would write
q = f"""
select ?director ?name where{{
wd:Q165325 wdt:P57 ?director .
?director <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('director', 'http://www.wikidata.org/entity/Q3772'), ('name', 'Quentin Tarantino')]


1

In [17]:
q = f"""
select distinct ?worker ?name where{{
wd:Q165325 wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58 ?worker .
?worker <http://schema.org/name> ?name
}}
order by asc(?name)
"""
run_query(q)

[('worker', 'http://www.wikidata.org/entity/Q15054445'), ('name', 'Ambrosia Kelley')]
[('worker', 'http://www.wikidata.org/entity/Q234094'), ('name', 'Chiaki Kuriyama')]
[('worker', 'http://www.wikidata.org/entity/Q207596'), ('name', 'Daryl Hannah')]
[('worker', 'http://www.wikidata.org/entity/Q187038'), ('name', 'David Carradine')]
[('worker', 'http://www.wikidata.org/entity/Q1071583'), ('name', 'Gordon Liu')]
[('worker', 'http://www.wikidata.org/entity/Q967797'), ('name', 'James Parks')]
[('worker', 'http://www.wikidata.org/entity/Q2075598'), ('name', 'Jonathan Loughran')]
[('worker', 'http://www.wikidata.org/entity/Q263178'), ('name', 'Julie Dreyfus')]
[('worker', 'http://www.wikidata.org/entity/Q11422725'), ('name', 'Jun Kunimura')]
[('worker', 'http://www.wikidata.org/entity/Q3811114'), ('name', 'Juri Manase')]
[('worker', 'http://www.wikidata.org/entity/Q1049366'), ('name', 'Kazuki Kitamura')]
[('worker', 'http://www.wikidata.org/entity/Q736403'), ('name', 'Kenji Ōba')]
[('worker

26

In [18]:
#point 4 is very general, let's start by counting the total number of workers that have participated in a Tarantino movie
subQuery = f"""
select distinct ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58 ?worker .
}}
"""
q = f"""
select (count(*) as ?n_distinct) where{{
    {{
        select distinct ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58 ?worker .

}}



    }}
}}
"""
run_query(q)

[('n_distinct', '274')]


1

In [19]:
#let's do for Woody Allen
subQuery = f"""
select distinct ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58 ?worker .
}}
"""
q = f"""
select (count(*) as ?n_distinct) where{{
    {{
        select distinct ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58 ?worker .

}}



    }}
}}
"""
run_query(q)

[('n_distinct', '894')]


1

In [20]:
#looking at the list of roles we have identified, we may have missed some roles. 
#Is it possible that there are properties related to workers in roles such as
#camera crew, costume designer, lighting, visual fx and so on? 
#let's expand the search of properties to the whole set of movies by Tarantino and Allen
#the object of the property will be a person, most likely, so we use the class Human
q = f"""
select distinct ?property ?name where{{
?movie ?property ?worker .
?property <http://schema.org/name> ?name .
?worker wdt:P31 wd:Q5 .
?movie wdt:P31 wd:Q11424 .
?movie wdt:P57 ?director .
filter (?director in (wd:Q3772, wd:Q25089))
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('property', 'http://www.wikidata.org/prop/direct/P2515'), ('name', 'costume designer')]
[('property', 'http://www.wikidata.org/prop/direct/P344'), ('name', 'director of photography')]
[('property', 'http://www.wikidata.org/prop/direct/P57'), ('name', 'director')]
[('property', 'http://www.wikidata.org/prop/direct/P58'), ('name', 'screenwriter')]
[('property', 'http://www.wikidata.org/prop/direct/P86'), ('name', 'composer')]
[('property', 'http://www.wikidata.org/prop/direct/P2554'), ('name', 'production designer')]
[('property', 'http://www.wikidata.org/prop/direct/P1431'), ('name', 'executive producer')]
[('property', 'http://www.wikidata.org/prop/direct/P187

16

In [22]:
#and let's redo the relevant queries to get the number of workers
subQuery = f"""
select distinct ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767
 ?worker .
}}
"""
q = f"""
select (count(*) as ?n_distinct) where{{
    {{
        select distinct ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767

 ?worker .

}}



    }}
}}
"""
run_query(q)

[('n_distinct', '286')]


1

In [23]:
subQuery = f"""
select distinct ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767
 ?worker .
}}
"""
q = f"""
select (count(*) as ?n_distinct) where{{
    {{
        select distinct ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767

 ?worker .

}}



    }}
}}
"""
run_query(q)

[('n_distinct', '919')]


1

## Task 4

In [25]:
#how many people worked in each tarantino movie?

#first get distinct movie and worker, assumption is that some people may have more than 1 role
subQuery = f"""
select distinct ?movie ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767
 ?worker

}}
"""

#this counts the number of workers for each movie
q = f"""
select ?movie ?name (count(?worker) as ?n_workers) where{{
?movie <http://schema.org/name> ?name .

    {{
        select distinct ?movie ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767

 ?worker



}}



    }}

}}
group by ?movie ?name
order by desc(?n_workers)
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q571032'), ('name', 'Django Unchained'), ('n_workers', '62')]
[('movie', 'http://www.wikidata.org/entity/Q153723'), ('name', 'Inglourious Basterds'), ('n_workers', '57')]
[('movie', 'http://www.wikidata.org/entity/Q47300912'), ('name', 'Once Upon a Time in Hollywood'), ('n_workers', '56')]
[('movie', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction'), ('n_workers', '43')]
[('movie', 'http://www.wikidata.org/entity/Q192115'), ('name', 'Sin City'), ('n_workers', '39')]
[('movie', 'http://www.wikidata.org/entity/Q165325'), ('name', 'Kill Bill Volume 1'), ('n_workers', '28')]
[('movie', 'http://www.wikidata.org/entity/Q431252'), ('name', 'Jackie Brown'), ('n_workers', '25')]
[('movie', 'http://www.wikidata.org/entity/Q1137310'), ('name', 'Death Proof'), ('n_workers', '24')]
[('movie', 'http://www.wikidata.org/entity/Q928019'), ('name', 'Kill Bill Volume 2'), ('n_workers', '24')]
[('movie', 'http://www.wikidata.org/entity/Q18225084

12

In [26]:
#how many different workers in each allen movie?

#first get distinct movie and worker, assumption is that some people may have more than 1 role
subQuery = f"""
select distinct ?movie ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767
 ?worker

}}
"""

#this counts the number of workers for each movie
q = f"""
select ?movie ?name (count(?worker) as ?n_workers) where{{
?movie <http://schema.org/name> ?name .

    {{
        select distinct ?movie ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767

 ?worker



}}



    }}

}}
group by ?movie ?name
order by desc(?n_workers)
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q1052826'), ('name', 'Celebrity'), ('n_workers', '74')]
[('movie', 'http://www.wikidata.org/entity/Q29658'), ('name', 'To Rome with Love'), ('n_workers', '48')]
[('movie', 'http://www.wikidata.org/entity/Q977372'), ('name', 'New York Stories'), ('n_workers', '46')]
[('movie', 'http://www.wikidata.org/entity/Q187423'), ('name', 'Zelig'), ('n_workers', '45')]
[('movie', 'http://www.wikidata.org/entity/Q3986392'), ('name', 'The Concert for New York City'), ('n_workers', '43')]
[('movie', 'http://www.wikidata.org/entity/Q551762'), ('name', 'Deconstructing Harry'), ('n_workers', '41')]
[('movie', 'http://www.wikidata.org/entity/Q910523'), ('name', 'Shadows and Fog'), ('n_workers', '40')]
[('movie', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris'), ('n_workers', '40')]
[('movie', 'http://www.wikidata.org/entity/Q1218888'), ('name', 'Love and Death'), ('n_workers', '40')]
[('movie', 'http://www.wikidata.org/entity/Q1004531'), 

51

In [27]:
#which tarantino movies used a narrator?
q = f"""
select distinct ?movie ?name ?narratorName where{{
?movie wdt:P31 wd:Q11424 ;
    <http://schema.org/name> ?name ;
    wdt:P57 wd:Q3772 ;
    wdt:P2438 ?narrator .
?narrator <http://schema.org/name> ?narratorName .
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q153723'), ('name', 'Inglourious Basterds'), ('narratorName', 'Samuel L. Jackson')]


1

In [28]:
#which allen movies used a narrator?
q = f"""
select distinct ?movie ?name ?narratorName where{{
?movie wdt:P31 wd:Q11424 ;
    <http://schema.org/name> ?name ;
    wdt:P57 wd:Q25089 ;
    wdt:P2438 ?narrator .
?narrator <http://schema.org/name> ?narratorName .
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q1146315'), ('name', 'Take the Money and Run'), ('narratorName', 'Jackson Beck')]


1

In [29]:
#the top 10 people that took part in a tarantino movie, ordered by number of appearances

#first get distinct movie and worker, assumption is that some people may have more than 1 role
subQuery = f"""
select distinct ?movie ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767
 ?worker

}}
"""

q = f"""
select ?worker ?name (count(*) as ?appearances) where{{

?worker <http://schema.org/name> ?name .

    {{
        select distinct ?movie ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767

 ?worker



}}



    }}

}}
group by ?worker ?name
order by desc(?appearances)
limit 10
"""
run_query(q)

[('worker', 'http://www.wikidata.org/entity/Q3772'), ('name', 'Quentin Tarantino'), ('appearances', '12')]
[('worker', 'http://www.wikidata.org/entity/Q273379'), ('name', 'Sally Menke'), ('appearances', '7')]
[('worker', 'http://www.wikidata.org/entity/Q220584'), ('name', 'Michael Madsen'), ('appearances', '7')]
[('worker', 'http://www.wikidata.org/entity/Q172678'), ('name', 'Samuel L. Jackson'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q514913'), ('name', 'Lawrence Bender'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q227123'), ('name', 'Zoë Bell'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q364017'), ('name', 'Robert Richardson'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q967797'), ('name', 'James Parks'), ('appearances', '5')]
[('worker', 'http://www.wikidata.org/entity/Q525318'), ('name', 'Michael Parks'), ('appearances', '4')]
[('worker', 'http://www.wikidata.org/entity/Q888311'), ('na

10

In [30]:
#of course, we should ignore the director, let's just extend the limit to 11
#first get distinct movie and worker, assumption is that some people may have more than 1 role
subQuery = f"""
select distinct ?movie ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767
 ?worker

}}
"""

q = f"""
select ?worker ?name (count(*) as ?appearances) where{{

?worker <http://schema.org/name> ?name .

    {{
        select distinct ?movie ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767

 ?worker



}}



    }}

}}
group by ?worker ?name
order by desc(?appearances)
limit 11
"""
run_query(q)

[('worker', 'http://www.wikidata.org/entity/Q3772'), ('name', 'Quentin Tarantino'), ('appearances', '12')]
[('worker', 'http://www.wikidata.org/entity/Q273379'), ('name', 'Sally Menke'), ('appearances', '7')]
[('worker', 'http://www.wikidata.org/entity/Q220584'), ('name', 'Michael Madsen'), ('appearances', '7')]
[('worker', 'http://www.wikidata.org/entity/Q172678'), ('name', 'Samuel L. Jackson'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q514913'), ('name', 'Lawrence Bender'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q227123'), ('name', 'Zoë Bell'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q364017'), ('name', 'Robert Richardson'), ('appearances', '6')]
[('worker', 'http://www.wikidata.org/entity/Q967797'), ('name', 'James Parks'), ('appearances', '5')]
[('worker', 'http://www.wikidata.org/entity/Q525318'), ('name', 'Michael Parks'), ('appearances', '4')]
[('worker', 'http://www.wikidata.org/entity/Q888311'), ('na

11

In [31]:
#for Allen
subQuery = f"""
select distinct ?movie ?worker where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767
 ?worker

}}
"""

q = f"""
select ?worker ?name (count(*) as ?appearances) where{{

?worker <http://schema.org/name> ?name .

    {{
        select distinct ?movie ?worker where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 ;

    wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P57|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767

 ?worker



}}



    }}

}}
group by ?worker ?name
order by desc(?appearances)
limit 11
"""
run_query(q)

[('worker', 'http://www.wikidata.org/entity/Q25089'), ('name', 'Woody Allen'), ('appearances', '51')]
[('worker', 'http://www.wikidata.org/entity/Q3831197'), ('name', 'Letty Aronson'), ('appearances', '22')]
[('worker', 'http://www.wikidata.org/entity/Q4726932'), ('name', 'Alisa Lepselter'), ('appearances', '21')]
[('worker', 'http://www.wikidata.org/entity/Q2223675'), ('name', 'Santo Loquasto'), ('appearances', '21')]
[('worker', 'http://www.wikidata.org/entity/Q273753'), ('name', 'Susan E. Morse'), ('appearances', '20')]
[('worker', 'http://www.wikidata.org/entity/Q2157353'), ('name', 'Robert Greenhut'), ('appearances', '19')]
[('worker', 'http://www.wikidata.org/entity/Q202725'), ('name', 'Mia Farrow'), ('appearances', '13')]
[('worker', 'http://www.wikidata.org/entity/Q376138'), ('name', 'Charles H. Joffe'), ('appearances', '13')]
[('worker', 'http://www.wikidata.org/entity/Q1006997'), ('name', 'Carlo Di Palma'), ('appearances', '11')]
[('worker', 'http://www.wikidata.org/entity/Q3

11

In [32]:
#In which movies did Tarantino both act and direct?
q = f"""
select distinct ?movie ?name where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P161 wd:Q3772 ;
    <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q18225084'), ('name', 'The Hateful Eight')]
[('movie', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction')]
[('movie', 'http://www.wikidata.org/entity/Q1137310'), ('name', 'Death Proof')]
[('movie', 'http://www.wikidata.org/entity/Q165325'), ('name', 'Kill Bill Volume 1')]
[('movie', 'http://www.wikidata.org/entity/Q431252'), ('name', 'Jackie Brown')]
[('movie', 'http://www.wikidata.org/entity/Q153723'), ('name', 'Inglourious Basterds')]
[('movie', 'http://www.wikidata.org/entity/Q72962'), ('name', 'Reservoir Dogs')]
[('movie', 'http://www.wikidata.org/entity/Q928019'), ('name', 'Kill Bill Volume 2')]
[('movie', 'http://www.wikidata.org/entity/Q571032'), ('name', 'Django Unchained')]


9

In [33]:
#In which movies did Allen both act and direct?
q = f"""
select distinct ?movie ?name where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P161 wd:Q25089 ;
    <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
[('movie', 'http://www.wikidata.org/entity/Q1354109'), ('name', 'Mighty Aphrodite')]
[('movie', 'http://www.wikidata.org/entity/Q729026'), ('name', 'Broadway Danny Rose')]
[('movie', 'http://www.wikidata.org/entity/Q740143'), ('name', 'Husbands and Wives')]
[('movie', 'http://www.wikidata.org/entity/Q806092'), ('name', 'Bananas')]
[('movie', 'http://www.wikidata.org/entity/Q614009'), ('name', 'Anything Else')]
[('movie', 'http://www.wikidata.org/entity/Q187423'), ('name', 'Zelig')]
[('movie', 'http://www.wikidata.org/entity/Q786585'), ('name', 'Sweet and Lowdown')]
[('movie', 'http://www.wikidata.org/entity/Q1218888'), ('name', 'Love and Death')]
[('movie', 'http://www.wikidata.org/entity/Q181555'), ('name', 'Crimes and Misdemeanors')]
[('movie', 'http://www.wikidata.org/entity/Q682002'), ('name', 'scoop')]
[('movie', 'http://www.wikidata.org/entity/Q653518'), ('name', 'Stardust Memories')]
[('mov

30

## Task 5

In [34]:
#Point 5. how many workers in Tarantino movies also worked in Allen movies?

#subquery used to select each worker only once
sub = f"""
select distinct ?worker where{{
?movie1 wdt:P31 wd:Q11424 .
?movie1 wdt:P57 wd:Q3772 .
?movie2 wdt:P31 wd:Q11424 .
?movie2 wdt:P57 wd:Q25089 .
#no director in those ors
?movie1 wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767 ?worker .
?movie2 wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767 ?worker

}}
"""
q = f"""
select (count(*) as ?n) where{{

    {{
        select distinct ?worker where{{

?movie1 wdt:P31 wd:Q11424 .

?movie1 wdt:P57 wd:Q3772 .

?movie2 wdt:P31 wd:Q11424 .

?movie2 wdt:P57 wd:Q25089 .

#no director in those ors

?movie1 wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767 ?worker .

?movie2 wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767 ?worker



}}



    }}

}}
"""
run_query(q)

[('n', '14')]


1

In [35]:
#there's only 14, let's list them
q = f"""
select distinct ?worker ?name where{{
?movie1 wdt:P31 wd:Q11424 .
?movie1 wdt:P57 wd:Q3772 .
?movie2 wdt:P31 wd:Q11424 .
?movie2 wdt:P57 wd:Q25089 .
#no director in those ors
?movie1 wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767 ?worker .
?movie2 wdt:P1040|wdt:P161|wdt:P162|wdt:P86|wdt:P58|wdt:P2515|wdt:P344|wdt:P2554|wdt:P1431|wdt:P2438|wdt:P175|wdt:P767 ?worker .
?worker <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('worker', 'http://www.wikidata.org/entity/Q185724'), ('name', 'Mike Myers')]
[('worker', 'http://www.wikidata.org/entity/Q203804'), ('name', 'Tim Roth')]
[('worker', 'http://www.wikidata.org/entity/Q104061'), ('name', 'Steve Buscemi')]
[('worker', 'http://www.wikidata.org/entity/Q231096'), ('name', 'Léa Seydoux')]
[('worker', 'http://www.wikidata.org/entity/Q109232'), ('name', 'Rosanna Arquette')]
[('worker', 'http://www.wikidata.org/entity/Q207596'), ('name', 'Daryl Hannah')]
[('worker', 'http://www.wikidata.org/entity/Q26806'), ('name', 'Danny DeVito')]
[('worker', 'http://www.wikidata.org/entity/Q36949'), ('name', 'Robert De Niro')]
[('worker', 'http://www.wikidata.org/entity/Q125017'), ('name', 'Uma Thurman')]
[('worker', 'http://www.wikidata.org/entity/Q185051'), ('name', 'Christopher Walken')]
[('worker', 'http://www.wikidata.org/entity/Q38111'), ('name', 'Leonardo DiCaprio')]
[('worker', 'http://www.wikidata.org/entity/Q202148'), ('name', 'Burt Reynolds')]
[('worker', 'http://

14

In [36]:
#Point 5. What is the film with the hightest number of shared actors?

#first select distinct all actors that have worked in movies from both directors
sub = f"""
select distinct ?actor where{{
?movie1 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 .
?movie1 wdt:P161 ?actor .
?movie2 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 .
?movie2 wdt:P161 ?actor .
}}
"""

q = f"""
select ?movie ?name (count(*) as ?n_shared) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 ?director ;
    wdt:P161 ?actor ;
    <http://schema.org/name> ?name .
filter(?director in (wd:Q25089, wd:Q3772))
}}
group by ?movie ?name
order by desc(?n_shared)
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q1052826'), ('name', 'Celebrity'), ('n_shared', '64')]
[('movie', 'http://www.wikidata.org/entity/Q153723'), ('name', 'Inglourious Basterds'), ('n_shared', '52')]
[('movie', 'http://www.wikidata.org/entity/Q571032'), ('name', 'Django Unchained'), ('n_shared', '52')]
[('movie', 'http://www.wikidata.org/entity/Q47300912'), ('name', 'Once Upon a Time in Hollywood'), ('n_shared', '51')]
[('movie', 'http://www.wikidata.org/entity/Q29658'), ('name', 'To Rome with Love'), ('n_shared', '45')]
[('movie', 'http://www.wikidata.org/entity/Q977372'), ('name', 'New York Stories'), ('n_shared', '39')]
[('movie', 'http://www.wikidata.org/entity/Q3986392'), ('name', 'The Concert for New York City'), ('n_shared', '39')]
[('movie', 'http://www.wikidata.org/entity/Q187423'), ('name', 'Zelig'), ('n_shared', '39')]
[('movie', 'http://www.wikidata.org/entity/Q910523'), ('name', 'Shadows and Fog'), ('n_shared', '35')]
[('movie', 'http://www.wikidata.org/entity/Q19211

63

In [37]:
#this result doesnt' convince me, let's scale down
#first select distinct all actors that have worked in movies from both directors
sub = f"""
select distinct ?actor ?name where{{
?movie1 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 .
?movie1 wdt:P161 ?actor .
?movie2 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 .
?movie2 wdt:P161 ?actor .

?actor <http://schema.org/name> ?name .
}}
"""
run_query(sub)

[('actor', 'http://www.wikidata.org/entity/Q185724'), ('name', 'Mike Myers')]
[('actor', 'http://www.wikidata.org/entity/Q203804'), ('name', 'Tim Roth')]
[('actor', 'http://www.wikidata.org/entity/Q104061'), ('name', 'Steve Buscemi')]
[('actor', 'http://www.wikidata.org/entity/Q231096'), ('name', 'Léa Seydoux')]
[('actor', 'http://www.wikidata.org/entity/Q109232'), ('name', 'Rosanna Arquette')]
[('actor', 'http://www.wikidata.org/entity/Q207596'), ('name', 'Daryl Hannah')]
[('actor', 'http://www.wikidata.org/entity/Q36949'), ('name', 'Robert De Niro')]
[('actor', 'http://www.wikidata.org/entity/Q125017'), ('name', 'Uma Thurman')]
[('actor', 'http://www.wikidata.org/entity/Q185051'), ('name', 'Christopher Walken')]
[('actor', 'http://www.wikidata.org/entity/Q38111'), ('name', 'Leonardo DiCaprio')]
[('actor', 'http://www.wikidata.org/entity/Q202148'), ('name', 'Burt Reynolds')]
[('actor', 'http://www.wikidata.org/entity/Q76819'), ('name', 'Christoph Waltz')]


12

In [39]:
sub = f"""
select distinct ?actor where{{
?movie1 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 .
?movie1 wdt:P161 ?actor .
?movie2 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 .
?movie2 wdt:P161 ?actor .
}}
"""
q = f"""
select ?film ?name (count(*) as ?n_shared) where{{
?film wdt:P161 ?actor .
?film wdt:P31 wd:Q11424 .
?film wdt:P57 ?director .
?film <http://schema.org/name> ?name .
filter(?director in (wd:Q3772, wd:Q25089)) .
    {{
        select distinct ?actor where{{

?movie1 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 .

?movie1 wdt:P161 ?actor .

?movie2 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 .

?movie2 wdt:P161 ?actor .

}}



    }}
}}
group by ?film ?name
order by desc(?n_shared)
"""
run_query(q)

[('film', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction'), ('n_shared', '5')]
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('name', 'The Concert for New York City'), ('n_shared', '4')]
[('film', 'http://www.wikidata.org/entity/Q47300912'), ('name', 'Once Upon a Time in Hollywood'), ('n_shared', '3')]
[('film', 'http://www.wikidata.org/entity/Q153723'), ('name', 'Inglourious Basterds'), ('n_shared', '3')]
[('film', 'http://www.wikidata.org/entity/Q977372'), ('name', 'New York Stories'), ('n_shared', '2')]
[('film', 'http://www.wikidata.org/entity/Q165325'), ('name', 'Kill Bill Volume 1'), ('n_shared', '2')]
[('film', 'http://www.wikidata.org/entity/Q928019'), ('name', 'Kill Bill Volume 2'), ('n_shared', '2')]
[('film', 'http://www.wikidata.org/entity/Q18126825'), ('name', 'Kill Bill: The Whole Bloody Affair'), ('n_shared', '2')]
[('film', 'http://www.wikidata.org/entity/Q571032'), ('name', 'Django Unchained'), ('n_shared', '2')]
[('film', 'http://www.wikidat

20

In [40]:
#which shared actors appear in pulp fiction?
sub = f"""
select distinct ?actor where{{
?movie1 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 .
?movie1 wdt:P161 ?actor .
?movie2 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 .
?movie2 wdt:P161 ?actor .
}}
"""
q = f"""
select ?actor ?name where{{
?film wdt:P161 ?a .
?film wdt:P31 wd:Q11424 .
?film wdt:P57 ?director .
?film <http://schema.org/name> ?filmName .
filter(?director in (wd:Q3772, wd:Q25089)) .
    {{
        select distinct ?actor where{{

?movie1 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 .

?movie1 wdt:P161 ?actor .

?movie2 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 .

?movie2 wdt:P161 ?actor .

}}



    }}
    filter(?a = ?actor) .
?a <http://schema.org/name> ?name .
filter regex(?filmName, 'pulp fiction', 'i') .

}}
group by ?film ?name
order by desc(?n_shared)
"""
run_query(q)

[('actor', 'http://www.wikidata.org/entity/Q185051'), ('name', 'Christopher Walken')]
[('actor', 'http://www.wikidata.org/entity/Q104061'), ('name', 'Steve Buscemi')]
[('actor', 'http://www.wikidata.org/entity/Q109232'), ('name', 'Rosanna Arquette')]
[('actor', 'http://www.wikidata.org/entity/Q203804'), ('name', 'Tim Roth')]
[('actor', 'http://www.wikidata.org/entity/Q125017'), ('name', 'Uma Thurman')]


5

In [41]:
#Point 5. actor most used by both directors
#interpretation of this query: among all actors that have worked both for Tarantino and Allen, return the one
#with the highest amount of appearances

#get all actors that worked for Tarantino
subT = f"""
select distinct ?movie1 ?actor1 where{{
?movie1 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 .
?movie1 wdt:P161 ?actor1 .
}}
"""

#get all actors that worked for Allen
subA = f"""
select distinct ?movie2 ?actor2 where{{
?movie2 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 .
?movie2 wdt:P161 ?actor2 .
}}
"""

#count how many times each actor has been used by Tarantino
countT = f"""
select ?actor1 (count(*) as ?n1) where{{
    
    {{
        select distinct ?movie1 ?actor1 where{{

?movie1 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 .

?movie1 wdt:P161 ?actor1 .

}}



    }}
}}
group by ?actor1
"""

#count how many times each actor has been used by Allen
countA = f"""
select ?actor2 (count(*) as ?n2) where{{
    
    {{
        select distinct ?movie2 ?actor2 where{{

?movie2 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 .

?movie2 wdt:P161 ?actor2 .

}}



    }}
}}
group by ?actor2
"""

#here we sum the appearances in tarantino movies with those in allen movies
q = f"""
select ?actor1 ?name ?total where{{
    {{
        select ?actor1 (count(*) as ?n1) where{{

    

    {{

        select distinct ?movie1 ?actor1 where{{

?movie1 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 .

?movie1 wdt:P161 ?actor1 .

}}




    }}

}}

group by ?actor1



    }}
    
    {{
        select ?actor2 (count(*) as ?n2) where{{

    

    {{

        select distinct ?movie2 ?actor2 where{{

?movie2 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 .

?movie2 wdt:P161 ?actor2 .

}}




    }}

}}

group by ?actor2



    }}
?actor1 <http://schema.org/name> ?name .
filter(?actor1 = ?actor2) .
bind((?n1 + ?n2) as ?total)
}}
group by ?actor1 ?name
order by desc(?total)
limit 1
"""
run_query(q)

[('actor1', 'http://www.wikidata.org/entity/Q125017'), ('name', 'Uma Thurman'), ('total', '5')]


1

In [42]:
#that was a long query, let's make sure the result is correct. in how many Tarantino movies did uma thurman act?
#get all actors that worked for Tarantino
subT = f"""
select distinct ?movie1 ?actor1 where{{
?movie1 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 .
?movie1 wdt:P161 ?actor1 .
}}
"""

#count how many times each actor has been used by Tarantino
countT = f"""
select ?actor1 ?name (count(*) as ?n1) where{{
    
    {{
        select distinct ?movie1 ?actor1 where{{

?movie1 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 .

?movie1 wdt:P161 ?actor1 .

}}



    }}
?actor1 <http://schema.org/name> ?name .
filter regex(?name, 'uma thurman', 'i')
}}
group by ?actor1 ?name
"""
run_query(countT)

[('actor1', 'http://www.wikidata.org/entity/Q125017'), ('name', 'Uma Thurman'), ('n1', '4')]


1

In [43]:
#in how many Allen movies did uma thurman act?
#get all actors that worked for Allen
subA = f"""
select distinct ?movie1 ?actor1 where{{
?movie1 wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 .
?movie1 wdt:P161 ?actor1 .
}}
"""

#count how many times each actor has been used by Allen
countA = f"""
select ?actor1 ?name (count(*) as ?n1) where{{
    
    {{
        select distinct ?movie1 ?actor1 where{{

?movie1 wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 .

?movie1 wdt:P161 ?actor1 .

}}



    }}
?actor1 <http://schema.org/name> ?name .
filter regex(?name, 'uma thurman', 'i')
}}
group by ?actor1 ?name
"""
run_query(countA)

[('actor1', 'http://www.wikidata.org/entity/Q125017'), ('name', 'Uma Thurman'), ('n1', '1')]


1

## Task 6

In [45]:
#Point 6. Is the maximum budget for a Tarantino movie higher than the max budged of an Allen movie?
t = f"""
select (max(?cost) as ?maxCost1) where{{
?movie wdt:P2130 ?cost ;
    wdt:P57 wd:Q3772 ;
    wdt:P31 wd:Q11424 .
}}
"""

a = f"""
select (max(?cost) as ?maxCost2) where{{
?movie wdt:P2130 ?cost ;
    wdt:P57 wd:Q25089 ;
    wdt:P31 wd:Q11424 .
}} 
"""

q = f"""
ask where{{

    {{
        select (max(?cost) as ?maxCost1) where{{

?movie wdt:P2130 ?cost ;

    wdt:P57 wd:Q3772 ;

    wdt:P31 wd:Q11424 .

}}



    }}
    
    {{
        select (max(?cost) as ?maxCost2) where{{

?movie wdt:P2130 ?cost ;

    wdt:P57 wd:Q25089 ;

    wdt:P31 wd:Q11424 .

}} 



    }}
    
    filter(?maxCost1 > ?maxCost2)

}}
"""
run_ask_query(q)

{'head': {'link': []}, 'boolean': True}

In [46]:
#let's print the maximum budget for a Tarantino movie
q = f"""
select ?movie ?name ?cost where{{
?movie wdt:P2130 ?cost ;
    wdt:P57 wd:Q3772 ;
    wdt:P31 wd:Q11424 ;
    <http://schema.org/name> ?name .
}}
group by ?movie ?name
order by desc(?cost)
limit 1
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q571032'), ('name', 'Django Unchained'), ('cost', '100000000')]


1

In [47]:
#let's print the maximum budget for an Allen movie
q = f"""
select ?movie ?name ?cost where{{
?movie wdt:P2130 ?cost ;
    wdt:P57 wd:Q25089 ;
    wdt:P31 wd:Q11424 ;
    <http://schema.org/name> ?name .
}}
group by ?movie ?name
order by desc(?cost)
limit 1
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q1321004'), ('name', 'You Will Meet a Tall Dark Stranger'), ('cost', '22000000')]


1

In [48]:
#another ask query. Does Tarantino have the highest grossing movie compared to Allen?
t = f"""
select (max(?gain) as ?maxGain1) where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q3772 ;
    wdt:P31 wd:Q11424 .
}}
"""

a = f"""
select (max(?gain) as ?maxGain2) where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q25089 ;
    wdt:P31 wd:Q11424 .
}} 
"""

q = f"""
ask where{{

    {{
        select (max(?gain) as ?maxGain1) where{{

?movie wdt:P2142 ?gain ;

    wdt:P57 wd:Q3772 ;

    wdt:P31 wd:Q11424 .

}}



    }}
    
    {{
        select (max(?gain) as ?maxGain2) where{{

?movie wdt:P2142 ?gain ;

    wdt:P57 wd:Q25089 ;

    wdt:P31 wd:Q11424 .

}} 



    }}
    
    filter(?maxGain1 > ?maxGain2)

}}
"""
run_ask_query(q)

{'head': {'link': []}, 'boolean': True}

In [49]:
#let's see the greatest amount of profit for the 2 directors
#let's print the maximum box office for a Tarantino movie
q = f"""
select ?movie ?name ?gain where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q3772 ;
    wdt:P31 wd:Q11424 ;
    <http://schema.org/name> ?name .
}}
group by ?movie ?name
order by desc(?gain)
limit 1
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q571032'), ('name', 'Django Unchained'), ('gain', '425368238')]


1

In [50]:
#let's print the maximum box office for an Allen movie
q = f"""
select ?movie ?name ?gain where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q25089 ;
    wdt:P31 wd:Q11424 ;
    <http://schema.org/name> ?name .
}}
group by ?movie ?name
order by desc(?gain)
limit 1
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris'), ('gain', '151100000')]


1

In [51]:
#Is the sum of box office for Tarantino movies greater than the sum of box office for Allen movies?
t = f"""
select (sum(?gain) as ?total1) where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q3772 ;
    wdt:P31 wd:Q11424 .
}}
"""

a = f"""
select (sum(?gain) as ?total2) where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q25089 ;
    wdt:P31 wd:Q11424 .
}} 
"""

q = f"""
ask where{{

    {{
        select (sum(?gain) as ?total1) where{{

?movie wdt:P2142 ?gain ;

    wdt:P57 wd:Q3772 ;

    wdt:P31 wd:Q11424 .

}}



    }}
    
    {{
        select (sum(?gain) as ?total2) where{{

?movie wdt:P2142 ?gain ;

    wdt:P57 wd:Q25089 ;

    wdt:P31 wd:Q11424 .

}} 



    }}
    
    filter(?total1 > ?total2)

}}
"""
run_ask_query(q)

{'head': {'link': []}, 'boolean': True}

In [52]:
#The total box office of Tarantino movies?
q = f"""
select (sum(?gain) as ?total) where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q3772 ;
    wdt:P31 wd:Q11424 .
}}
"""
run_query(q)

[('total', '2054698758')]


1

In [53]:
#The total box office of Allen movies?
q = f"""
select (sum(?gain) as ?total) where{{
?movie wdt:P2142 ?gain ;
    wdt:P57 wd:Q25089 ;
    wdt:P31 wd:Q11424 .
}}
"""
run_query(q)

[('total', '660479723')]


1

## Task 7.1

In [54]:
#Point 7. Who has the film with more nominations for Academy Awards?

#property "nominated for" applies to both directors and movies, let's check the nominations for Kill Bill, if any
q = f"""
select ?award ?name where{{
wd:Q165325 wdt:P1411 ?award .
?award <http://schema.org/name> ?name
}}
"""
run_query(q)

[('award', 'http://www.wikidata.org/entity/Q1377772'), ('name', 'European Film Award for Best Non-European Film')]


1

In [55]:
#that doesn't look like an academy award, let's use the whole tarantino filmography
q = f"""
select ?movieName ?award ?name where{{
?movie wdt:P1411 ?award .
?movie wdt:P31 wd:Q11424 .
?movie wdt:P57 wd:Q3772 .
?movie <http://schema.org/name> ?movieName .
?award <http://schema.org/name> ?name
}}
order by asc(?movieName)
"""
run_query(q)

[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q41417'), ('name', 'Academy Award for Best Writing, Original Screenplay')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q106291'), ('name', 'Academy Award for Best Supporting Actor')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q131520'), ('name', 'Academy Award for Best Cinematography')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q488645'), ('name', 'Academy Award for Best Sound Editing')]
[('movieName', 'Inglourious Basterds'), ('award', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture')]
[('movieName', 'Inglourious Basterds'), ('award', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('movieName', 'In

63

In [56]:
#maybe it's better to check only the awards, without the movie
q = f"""
select distinct ?award ?name where{{
?movie wdt:P1411 ?award .
?movie wdt:P31 wd:Q11424 .
?movie wdt:P57 wd:Q3772 .
?award <http://schema.org/name> ?name
}}
order by asc(?movieName)
"""
run_query(q)

[('award', 'http://www.wikidata.org/entity/Q3404523'), ('name', 'David di Donatello for Best Foreign Actor')]
[('award', 'http://www.wikidata.org/entity/Q165313'), ('name', 'National Society of Film Critics Award for Best Supporting Actor')]
[('award', 'http://www.wikidata.org/entity/Q787145'), ('name', 'BAFTA Award for Best Editing')]
[('award', 'http://www.wikidata.org/entity/Q1011509'), ('name', 'Golden Globe Award for Best Motion Picture – Drama')]
[('award', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture')]
[('award', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('award', 'http://www.wikidata.org/entity/Q106301'), ('name', 'Academy Award for Best Supporting Actress')]
[('award', 'http://www.wikidata.org/entity/Q139184'), ('name', 'BAFTA Award for Best Film')]
[('award', 'http://www.wikidata.org/entity/Q281939'), ('name', 'Academy Award for Best Film Editing')]
[('award', 'http://www.wikidata.org/entit

50

In [57]:
#we see that there are various academy awards, let's see only those
q = f"""
select ?movieName ?award ?name where{{
?movie wdt:P1411 ?award .
?movie wdt:P31 wd:Q11424 .
?movie wdt:P57 wd:Q3772 .
?movie <http://schema.org/name> ?movieName .
?award <http://schema.org/name> ?name .
filter regex(?name, '.*academy award.*', 'i')
}}
order by asc(?movieName)
"""
run_query(q)

[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q41417'), ('name', 'Academy Award for Best Writing, Original Screenplay')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q106291'), ('name', 'Academy Award for Best Supporting Actor')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q131520'), ('name', 'Academy Award for Best Cinematography')]
[('movieName', 'Django Unchained'), ('award', 'http://www.wikidata.org/entity/Q488645'), ('name', 'Academy Award for Best Sound Editing')]
[('movieName', 'Inglourious Basterds'), ('award', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture')]
[('movieName', 'Inglourious Basterds'), ('award', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('movieName', 'In

24

In [58]:
AcademyDirector = 'wd:Q103360'
#let's count how many Academy Awards for best director Tarantino has from his movies
q = f"""
select (count(*) as ?nominations) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1411 wd:Q103360 .
}}
"""
run_query(q)

[('nominations', '2')]


1

In [59]:
#let's see for which movies he was nominated best director
q = f"""
select ?movie ?name where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    <http://schema.org/name> ?name ;
    wdt:P1411 wd:Q103360 .
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q104123'), ('name', 'Pulp Fiction')]
[('movie', 'http://www.wikidata.org/entity/Q153723'), ('name', 'Inglourious Basterds')]


2

In [60]:
#what about Woody Allen?
q = f"""
select (count(*) as ?nominations) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1411 wd:Q103360 .
}}
"""
run_query(q)

[('nominations', '7')]


1

In [61]:
#let's see for which movies he was nominated best director
q = f"""
select ?movie ?name where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    <http://schema.org/name> ?name ;
    wdt:P1411 wd:Q103360 .
}}
"""
run_query(q)

[('movie', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris')]
[('movie', 'http://www.wikidata.org/entity/Q1004531'), ('name', 'Bullets Over Broadway')]
[('movie', 'http://www.wikidata.org/entity/Q729026'), ('name', 'Broadway Danny Rose')]
[('movie', 'http://www.wikidata.org/entity/Q181555'), ('name', 'Crimes and Misdemeanors')]
[('movie', 'http://www.wikidata.org/entity/Q233464'), ('name', 'Annie Hall')]
[('movie', 'http://www.wikidata.org/entity/Q845057'), ('name', 'Hannah and Her Sisters')]
[('movie', 'http://www.wikidata.org/entity/Q961403'), ('name', 'Interiors')]


7

In [62]:
#what class is AcademyDirector?
q = f"""
select ?class ?name where{{
wd:Q103360 wdt:P31 ?class .
?class <http://schema.org/name> ?name
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q96474679'), ('name', 'award for best direction')]
[('class', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]


2

## Task 7

In [64]:
#how many academy award nominations did Tarantino movies get?
q = f"""
select (count(*) as ?nominations) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1411 ?award .
?award wdt:P31 wd:Q19020
}}
"""
run_query(q)

[('nominations', '24')]


1

In [65]:
#how many academy award nominations did Allen movies get?
q = f"""
select (count(*) as ?nominations) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1411 ?award .
?award wdt:P31 wd:Q19020
}}
"""
run_query(q)

[('nominations', '52')]


1

In [66]:
#in total Allen's movies have received more Academy Award nominations. We can also use an ask query
q1 = f"""
select (count(*) as ?nominations1) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1411 ?award .
?award wdt:P31 wd:Q19020
}}
"""

q2 = f"""
select (count(*) as ?nominations2) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1411 ?award .
?award wdt:P31 wd:Q19020
}}
"""

#does Allen have more Academy Award nominations for the movies he directed?
q = f"""
ask where{{
    {{
        select (count(*) as ?nominations1) where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P1411 ?award .

?award wdt:P31 wd:Q19020

}}



    }}
    
    {{
        select (count(*) as ?nominations2) where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 ;

    wdt:P1411 ?award .

?award wdt:P31 wd:Q19020

}}



    }}
    
    filter(?nominations2 > ?nominations1)
}}
"""
run_ask_query(q)

{'head': {'link': []}, 'boolean': True}

In [67]:
#Point 7. Who has films with more nominations for Academy Awards and who won more Academy Awards?

#let's see which tarantino movies won academy awards
q = f"""
select ?name ?awardName where{{
?movie wdt:P31 wd:Q11424;
    <http://schema.org/name> ?name ;
    wdt:P57 wd:Q3772 ;
    wdt:P166 ?award .
?award wdt:P31 wd:Q19020 ;
    <http://schema.org/name> ?awardName
}}
"""
run_query(q)

[('name', 'The Hateful Eight'), ('awardName', 'Academy Award for Best Original Score')]
[('name', 'Pulp Fiction'), ('awardName', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Inglourious Basterds'), ('awardName', 'Academy Award for Best Supporting Actor')]
[('name', 'Django Unchained'), ('awardName', 'Academy Award for Best Supporting Actor')]
[('name', 'Django Unchained'), ('awardName', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Once Upon a Time in Hollywood'), ('awardName', 'Academy Award for Best Production Design')]
[('name', 'Once Upon a Time in Hollywood'), ('awardName', 'Academy Award for Best Supporting Actor')]


7

In [68]:
#let's see which allen movies won academy awards
q = f"""
select ?name ?awardName where{{
?movie wdt:P31 wd:Q11424;
    <http://schema.org/name> ?name ;
    wdt:P57 wd:Q25089 ;
    wdt:P166 ?award .
?award wdt:P31 wd:Q19020 ;
    <http://schema.org/name> ?awardName
}}
"""
run_query(q)

[('name', 'Midnight in Paris'), ('awardName', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Bullets Over Broadway'), ('awardName', 'Academy Award for Best Supporting Actress')]
[('name', 'Mighty Aphrodite'), ('awardName', 'Academy Award for Best Supporting Actress')]
[('name', 'Blue Jasmine'), ('awardName', 'Academy Award for Best Actress')]
[('name', 'Vicky Cristina Barcelona'), ('awardName', 'Academy Award for Best Supporting Actress')]
[('name', 'Annie Hall'), ('awardName', 'Academy Award for Best Director')]
[('name', 'Annie Hall'), ('awardName', 'Academy Award for Best Writing, Original Screenplay')]
[('name', 'Annie Hall'), ('awardName', 'Academy Award for Best Actress')]
[('name', 'Annie Hall'), ('awardName', 'Academy Award for Best Picture')]
[('name', 'Hannah and Her Sisters'), ('awardName', 'Academy Award for Best Supporting Actress')]
[('name', 'Hannah and Her Sisters'), ('awardName', 'Academy Award for Best Supporting Actor')]
[('name', 'Hannah and Her 

12

In [73]:
#woody allen won more academy awards through his movies, let's make an ask query to check this fact formally

#count how many allen movies won academy awards
q1 = f"""
select (count(*) as ?allenAwardWinners) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P166 ?award .

?award wdt:P31 wd:Q19020 .

}}
"""

#count how many tarantino movies won academy awards
q2 = f"""
select (count(*) as ?tarantinoAwardWinners) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P166 ?award .

?award wdt:P31 wd:Q19020 .

}}
"""

#did allen movies receive a greater total of academy awards with respect to tarantino movies?
q = f"""
ask where{{

    {{
        select (count(*) as ?allenAwardWinners) where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 ;

    wdt:P166 ?award .



?award wdt:P31 wd:Q19020 .



}}



    }}
    
    {{
        select (count(*) as ?tarantinoAwardWinners) where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P166 ?award .



?award wdt:P31 wd:Q19020 .



}}



    }}
    
    filter(?allenAwardWinners > ?tarantinoAwardWinners)
}}
"""

run_ask_query(q)

{'head': {'link': []}, 'boolean': False}

In [74]:
#what about nominations?

#run the same ask query but for nominations

q1 = f"""
select (count(*) as ?allenNominations) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q25089 ;
    wdt:P1411 ?award .

?award wdt:P31 wd:Q19020 .

}}
"""

q2 = f"""
select (count(*) as ?tarantinoNominations) where{{
?movie wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P1411 ?award .

?award wdt:P31 wd:Q19020 .

}}
"""

q = f"""
ask where{{

    {{
        select (count(*) as ?allenNominations) where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q25089 ;

    wdt:P1411 ?award .



?award wdt:P31 wd:Q19020 .



}}



    }}
    
    {{
        select (count(*) as ?tarantinoNominations) where{{

?movie wdt:P31 wd:Q11424 ;

    wdt:P57 wd:Q3772 ;

    wdt:P1411 ?award .



?award wdt:P31 wd:Q19020 .



}}



    }}
    
    filter(?allenNominations > ?tarantinoNominations)
}}
"""

run_ask_query(q)

{'head': {'link': []}, 'boolean': True}